In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data = pd.read_csv("/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data.describe()

1) Мы работаем с набором данных, который сожержит различную составную информацию о красных винах, а также об их качестве.

Постановка задачи: Проблема заключается в том, чтобы автоматически прогнозировать качество на основе этой информации, чтобы помочь улучшить производство вина и удовлетворить вкусы потребителей на рынках.

In [ ]:
data.dtypes

Наш target - это quality - качество красного вина по шкале (значения от 1 до 10). Соответственно, можем сделать вывод, что наш целевой признак является ранговым. Поэтому можем рассматривать задачу как классификацию, так и регрессию.

In [ ]:
plt.figure(figsize = (12,12))
sns.heatmap(data = data.corr(), annot = True, square = True, cbar = True)

Из-за того что наши значения находятся в разных диапазонах, мы можем масштабировать данные.

In [ ]:
sc = StandardScaler()
X2 = data.drop(['quality'], axis = 1)
y = data['quality']
X = sc.fit_transform(X2)

Разбиваем наш набор данных на обучающую (80%) и валидационную (20%) выборки

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [ ]:
clfTre = tree.DecisionTreeClassifier(max_depth=3)
clfTre.fit(x_train, y_train)
#y_pred = knn.predict(x_test)
#accuracy_score(y_test, y_pred)

Обучаем алгоритм классификации DecisionTreeClassifier и оцениваем качество каждой модели на отложенной выборке.

In [ ]:
clfTre = tree.DecisionTreeClassifier(max_depth=3)
clfTre.fit(x_train, y_train)
y_pred = clfTre.predict(x_test)
accuracy_score(y_test, y_pred)

In [ ]:
clfTre = tree.DecisionTreeClassifier(max_depth=3)
clfTre.fit(x_train, y_train)
y_pred = clfTre.predict(x_test)
mean_squared_error(y_test, y_pred)

2)Настройка гиперпараметров дерева

Создаем генератор разбиения и осуществляем кросс валидацию модели

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
tree_params = {"criterion": ["gini", "entropy"], "max_depth": list(range(2,30,1)), 
                  "min_samples_leaf": list(range(5,20,1))}
grid_tree = GridSearchCV(tree.DecisionTreeClassifier(), tree_params, cv=kf)
grid_tree.fit(x_train, y_train)
tree_clf = grid_tree.best_estimator_
print(grid_tree.best_estimator_)
print(grid_tree.best_score_)


Строим валидационные кривые по гипермараметрам 

In [ ]:
from yellowbrick.model_selection import ValidationCurve

viz = ValidationCurve(
    tree_clf, param_name="max_depth",
    param_range=np.arange(1, 11), cv=10, scoring="r2")
viz.fit(X, y)

In [ ]:
from yellowbrick.model_selection import ValidationCurve

viz = ValidationCurve(
    tree_clf, param_name="min_samples_leaf",
    param_range=np.arange(1, 11), cv=10, scoring="r2")
viz.fit(X, y)

In [ ]:
Строим наше дерево решений

In [ ]:
import graphviz 
dot_data = tree.export_graphviz(tree_clf, out_file=None, max_depth=3, feature_names=list(X2.columns.values), filled=True, rounded=True)
valgTre = graphviz.Source(dot_data) 
valgTre

Оцениваем и визуализируем важность признаков

In [ ]:
importnce = tree_clf.feature_importances_
plt.figure(figsize=(10,10))
plt.title("Feature Importances")
plt.barh(X2.columns, importnce, align="center")

Вывод: 


Можем видеть что нам необходимо сосредоточиться на отношениях качества алкоголя и плотности алкоголя, чтобы получить содержательное исследование.

3)Строим модель случайного леса (Random Forest) на отложенной выборке

In [ ]:
forest_params = {"bootstrap":[True, False], "max_depth": list(range(2,10,1)), 
                  "min_samples_leaf": list(range(5,20,1))}
grid_forest = GridSearchCV(RandomForestClassifier(n_estimators=10), forest_params, cv=kf)
grid_forest.fit(x_train, y_train)
forest_clf = grid_forest.best_estimator_
print(grid_forest.best_estimator_)
print(grid_forest.best_score_)

In [ ]:
from yellowbrick.model_selection import ValidationCurve

viz = ValidationCurve(
    forest_clf, param_name="max_depth",
    param_range=np.arange(1, 11), cv=10, scoring="r2")
viz.fit(X, y)

In [ ]:
from yellowbrick.model_selection import ValidationCurve

viz = ValidationCurve(
    forest_clf, param_name="min_samples_leaf",
    param_range=np.arange(1, 11), cv=10, scoring="r2")
viz.fit(X, y)

In [ ]:
importnce = forest_clf.feature_importances_
plt.figure(figsize=(10,10))
plt.title("Feature Importances")
plt.barh(X2.columns, importnce, align="center")

Вывод: 


Из всех алгоритмов, случайный лес дал лучшую модель, а точность означает, что этот алгоритмы правильно предсказывают тестовые данные. После анализа этого набора данных, некоторые особенности имеют большее влияние на определение качества вина, есть некоторое представление о критериях качества вина:


*Алкоголь является наиболее важным фактором, определяющим качество вина. Если процент алкоголя достаточно высок, значит, качество вина должно быть лучше.

*Сульфаты - еще один критерий отбора хороших вин, с высоким содержанием сульфатов качество вина повышается.

*Лимонная кислота - еще один критерий выбора, она должна быть выше, чтобы выбрать более лучшее вино.

При этом:

*Летучая кислотность в хорошем вине должна быть меньше.

*Диоксид серы - это еще один фактор, снижающий качество вина.

